## Homework

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
PATH = '/content/drive/MyDrive/YCup/'

In [4]:
data = []

with open(PATH + 'test') as f:
    for line in f.readlines():
        data.append([id for id in line.split()])

data = pd.DataFrame(np.array(data, dtype=object), columns=['tracks'])
data.reset_index(inplace=True)
data = data.explode('tracks', ignore_index=True)
data.rename(columns={'index': 'user_id', 'tracks': 'track_id'},
          inplace=True)
data['order'] = data.groupby('user_id').cumcount(ascending=False)

In [5]:
data

,user_id,track_id,order
0,0,454758,19
1,0,382341,18
2,0,240893,17
3,0,280388,16
4,0,362253,15
...,...,...,...
23262195,289913,448288,4
23262196,289913,1343,3
23262197,289913,86420,2
23262198,289913,186436,1


In [6]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, user_column='user_id', rank_column='order', random_seed=23):
        self.n_folds = n_folds
        self.p = p
        self.seed = random_seed
        self.user_column = user_column
        self.rank_column = rank_column
    
    def split(self, df):
        users = df[self.user_column].unique()
        users_count = len(users)
        
        np.random.seed(self.seed)
        np.random.shuffle(users)
        
        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            test_mask = df[self.user_column].isin(test_fold_users) & (df['order'] < self.p)
            train_mask = ~df[self.user_column].isin(test_fold_users)
            
            yield train_mask, test_mask

In [7]:
p = 3
cv = UsersKFoldPOut(n_folds=4, p=p)

for i, (train_mask, test_mask) in enumerate(cv.split(data)):
    train = data[train_mask]
    test = data[test_mask]
    assert set(train['user_id'].unique()).intersection(test['user_id'].unique()) == set(), 'Между train и test есть общие пользователи'
    assert np.all(test['order'] < p), 'В test более p последних треков'
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 17456794, Test: 217437
Fold#1 | Train: 17456794, Test: 217437
Fold#2 | Train: 17456885, Test: 217434
Fold#3 | Train: 17456885, Test: 217434
